In [ ]:
import cv2
import numpy as np
import imutils
import tqdm
import os
from moviepy.editor import ImageSequenceClip
from random import randint
from copy import deepcopy
import time
from scipy.spatial import cKDTree
#import tensorflow as tf
#print(tf.test.gpu_device_name())

class VideoStitcher:
    def __init__(self, left_video_in_path,
                 right_video_in_path,
                 left_video_in_path2,
                 right_video_in_path2,
                 video_out_path, video_out_width=800,
                 display=True):
        # Initialize arguments
        self.left_video_in_path = left_video_in_path
        self.right_video_in_path = right_video_in_path
        self.left_video_in_path2 = left_video_in_path2
        self.right_video_in_path2 = right_video_in_path2
        self.video_out_path = video_out_path
        self.video_out_width = video_out_width
        self.display = display

        # Initialize the saved homography matrix
        self.saved_homo_matrix1 = None
        self.saved_homo_matrix2 = None
        self.saved_homo_matrix3 = None

    def stitch(self, images, ratio=0.75, reproj_thresh=4.0):
        # Unpack the images
        (image_d, image_c, image_b, image_a) = images
        
        image_dd = cv2.cvtColor(image_d, cv2.COLOR_RGB2GRAY)
        image_cc = cv2.cvtColor(image_c, cv2.COLOR_RGB2GRAY)
        image_bb = cv2.cvtColor(image_b, cv2.COLOR_RGB2GRAY)
        image_aa = cv2.cvtColor(image_a, cv2.COLOR_RGB2GRAY)

        # If the saved homography matrix is None, then we need to apply keypoint matching to construct it
        if self.saved_homo_matrix1 is None:
            # Detect keypoints and extract
            tfirst= time.time()
            (keypoints_a, features_a) = self.detect_and_extract(image_aa)
            print(time.time()-tfirst)
            (keypoints_b, features_b) = self.detect_and_extract(image_bb)
            tfirst1= time.time()
            matched_keypoints1 = self.match_keypoints(keypoints_a, keypoints_b, features_a, features_b, ratio, reproj_thresh)
            print(time.time()-tfirst1)
            if matched_keypoints1  is None:
                return None
            self.saved_homo_matrix1 = matched_keypoints1[1]
            print('h1')
            
            
        hmgphy1 = self.saved_homo_matrix1
        
        if self.saved_homo_matrix2 is None:
            (keypoints_c, features_c) = self.detect_and_extract(image_cc)
            matched_keypoints2 = self.match_keypoints(keypoints_b, keypoints_c, features_b, features_c, ratio, reproj_thresh)

            if matched_keypoints2  is None:
                return None
            self.saved_homo_matrix2 = matched_keypoints2[1]
            print('h2')
            
        hmgphy2 = self.saved_homo_matrix2
        
        if self.saved_homo_matrix3 is None:
            (keypoints_d, features_d) = self.detect_and_extract(image_dd)
            matched_keypoints3 = self.match_keypoints(keypoints_c, keypoints_d, features_c, features_d, ratio, reproj_thresh)

            if matched_keypoints3  is None:
                return None
            self.saved_homo_matrix3 = matched_keypoints3[1]
            print('h3')
            
        hmgphy3 = self.saved_homo_matrix3
            
        output_shape1 = (image_a.shape[1] + image_b.shape[1], image_a.shape[0])
        resultfab = cv2.warpPerspective(image_a, hmgphy1, output_shape1)
        resultfab[0:image_b.shape[0], 0:image_b.shape[1]] = image_b
        
        #cv2.imwrite("separate/12.jpg", cv2.cvtColor(resultfab, cv2.COLOR_BGR2RGB ))
        
        output_shape2 = (resultfab.shape[1] + image_c.shape[1], resultfab.shape[0])
        resultfabc = cv2.warpPerspective(resultfab, hmgphy2, output_shape2)
        resultfabc[0:image_c.shape[0], 0:image_c.shape[1]] = image_c
        
        #cv2.imwrite("separate/123.jpg", cv2.cvtColor(resultfabc, cv2.COLOR_BGR2RGB ))
        
        output_shape3 = (resultfabc.shape[1] + image_d.shape[1], resultfabc.shape[0])
        resultfabcd = cv2.warpPerspective(resultfabc, hmgphy3, output_shape3)
        resultfabcd[0:image_d.shape[0], 0:image_d.shape[1]] = image_d
        
        #cv2.imwrite("separate/1234.jpg", cv2.cvtColor(resultfabcd, cv2.COLOR_BGR2RGB ))
        #resultfabcd=(resultfabcd/255.0)
        return resultfabcd
    


    @staticmethod
    def detect_and_extract(image):
        # Detect and extract features from the image (DoG keypoint detector and SIFT feature extractor)
        #descriptor = cv2.xfeatures2d.SIFT_create()
        
        #addd start
        #detector = cv2.FeatureDetector_create("SIFT")## original
        #detector = cv2.ORB_create(nfeatures=8000, scaleFactor=1.1, nlevels=8, edgeThreshold=10, firstLevel=0, WTA_K=2, patchSize=10)

        algo = cv2.xfeatures2d.SIFT_create( nOctaveLayers=3, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6)
        #detector = cv2.SIFT_create()
        #detector = cv2.xfeatures2d.SIFT_create()
        #detector = cv2.xfeatures2d.SURF_create()

        #descriptor = cv2.DescriptorExtractor_create("SIFT")###original
        #descriptor = cv2.xfeatures2d.SIFT_create()
        
        #keypoints = algo.detect(image)
        #keypoints, features = algo.compute(image, keypoints)
        
        eps = 1e-7
        keypoints = algo.detect(image)
        keypoints, features = algo.compute(image, keypoints)
        features /= (features.sum(axis=1, keepdims=True) + eps)
        features = np.sqrt(features)



        #(keypoints, features) = descriptor.detectAndCompute(image, None)

        # Convert the keypoints from KeyPoint objects to numpy arrays
        keypoints = np.float32([keypoint.pt for keypoint in keypoints])

        # Return a tuple of keypoints and features
        return (keypoints, features)

    @staticmethod
    def match_keypoints(keypoints_a, keypoints_b, features_a, features_b, ratio, reproj_thresh):
        # Compute the raw matches and initialize the list of actual matches
        #matcher = cv2.DescriptorMatcher_create("BruteForce")
        #matcher = cv.BFMatcher()
        ################
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks=50)   # or pass empty dictionary
        flann = cv2.FlannBasedMatcher(index_params,search_params)
        flann_matches = flann.knnMatch(features_a, features_b, k=2)
        print(" flann_matches raw matches with rootSift: ",len(flann_matches))
        #print(flann_matches)
        #########################
        bf = cv2.BFMatcher()
        bf_matches = bf.knnMatch(features_a,features_b,k=2)
        print(" BF raw matches with rootSift: ",len(bf_matches))
        #print(type(bf_matches))
        #print(bf_matches)
        raw_matches = np.concatenate([flann_matches, bf_matches],axis=0)
        #in_first = set(flann_matches)
        #in_second = set(bf_matches)
        #in_second_but_not_in_first = in_second - in_first
        #result = first_list + list(in_second_but_not_in_first)
        #pA=[[0,0,0],[0,1,0],[1,2,4],[10,3,4],[1,20,1],[5,3,2]]
        #pB=[[14,1,0],[1,2,4],[1,20,1],[15,1,0]]
        #kdtree = cKDTree(flann_matches)
      
        matches = []

        for raw_match in raw_matches:
            # Ensure the distance is within a certain ratio of each other (i.e. Lowe's ratio test)
            if len(raw_match) == 2 and raw_match[0].distance < raw_match[1].distance * ratio:
                matches.append((raw_match[0].trainIdx, raw_match[0].queryIdx))
        
        print(" combine good matches with rootSift: ",len(matches))
        
        matches = list(set(matches))
        
        print(" combine good matches with rootSift unique value : ", len(matches))

        # Computing a homography requires at least 4 matches
        if len(matches) > 4:
            # Construct the two sets of points
            points_a = np.float32([keypoints_a[i] for (_, i) in matches])
            points_b = np.float32([keypoints_b[i] for (i, _) in matches])

            # Compute the homography between the two sets of points
            (homography_matrix, status) = cv2.findHomography(points_a, points_b, cv2.RANSAC, reproj_thresh)

            # Return the matches, homography matrix and status of each matched point
            return (matches, homography_matrix, status)

        # No homography could be computed
        return None

    @staticmethod
    def draw_matches(image_a, image_b, keypoints_a, keypoints_b, matches, status):
        # Initialize the output visualization image
        (height_a, width_a) = image_a.shape[:2]
        (height_b, width_b) = image_b.shape[:2]
        visualisation = np.zeros((max(height_a, height_b), width_a + width_b, 3), dtype="uint8")
        visualisation[0:height_a, 0:width_a] = image_a
        visualisation[0:height_b, width_a:] = image_b

        for ((train_index, query_index), s) in zip(matches, status):
            # Only process the match if the keypoint was successfully matched
            if s == 1:
                # Draw the match
                point_a = (int(keypoints_a[query_index][0]), int(keypoints_a[query_index][1]))
                point_b = (int(keypoints_b[train_index][0]) + width_a, int(keypoints_b[train_index][1]))
                cv2.line(visualisation, point_a, point_b, (0, 255, 0), 1)

        # return the visualization
        
        
        return visualisation

    def run(self):
        # Set up video capture
        left_video = cv2.VideoCapture(self.left_video_in_path)
        if left_video is None:
            print(" no left_video ")
           
        right_video = cv2.VideoCapture(self.right_video_in_path)
        if right_video is None:
            print(" no right_video ")
          
        left_video2 = cv2.VideoCapture(self.left_video_in_path2)
        if left_video2 is None:
            print(" no left_video2 ")
           
        
        right_video2 = cv2.VideoCapture(self.right_video_in_path2)
        
        if right_video2 is None:
            print(" no right_video2")
            
            
        print('[INFO]: {} and {} and {} and {} loaded'.format(self.left_video_in_path.split('/')[-1],
                                                              self.right_video_in_path.split('/')[-1],
                                                              self.left_video_in_path2.split('/')[-1],
                                                              self.right_video_in_path2.split('/')[-1]))
        print('[INFO]: Video stitching starting....')

        # Get information about the videos
        n_frames = min(int(left_video.get(cv2.CAP_PROP_FRAME_COUNT)),
                       int(right_video.get(cv2.CAP_PROP_FRAME_COUNT)),
                       int(left_video2.get(cv2.CAP_PROP_FRAME_COUNT)),
                       int(right_video2.get(cv2.CAP_PROP_FRAME_COUNT)))
        
        #fps = int(left_video.get(cv2.CAP_PROP_FPS))
        fps=15
        frames = []

        for _ in tqdm.tqdm(np.arange(n_frames)):
            # Grab the frames from their respective video streams
            ok, img1 = left_video.read()
            _, img2 = right_video.read()
            ok2, img3 = left_video2.read()
            _, img4 = right_video2.read()
            
            
            if ok & ok2 :
                # Stitch the frames together to form the panorama
                #left = left.astype('uint8')

              
                img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
                #cv2.imwrite("separate/f1.jpg", cv2.cvtColor(img1, cv2.COLOR_BGR2RGB ))
                
                img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
                #cv2.imwrite("separate/f2.jpg", cv2.cvtColor(img2, cv2.COLOR_BGR2RGB ))
                
                img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)
                #cv2.imwrite("separate/f3.jpg", cv2.cvtColor(img3, cv2.COLOR_BGR2RGB ))
                
                img4 = cv2.cvtColor(img4, cv2.COLOR_BGR2RGB)
                #cv2.imwrite("separate/f4.jpg", cv2.cvtColor(img4, cv2.COLOR_BGR2RGB ))
                

                stitched_frame = self.stitch([img1, img2, img3, img4])
                
        
                # No homography could not be computed
                if stitched_frame is None:
                    print("[INFO]: Homography could not be computed!")
                    break

                # Add frame to video
                #stitched_frame1 = stitched_frame1.astype('uint8')
                #stitched_frame2 = stitched_frame2.astype('uint8')
                
                
                #print(stitched_frame.shape)
                stitched_frame = imutils.resize(stitched_frame, width=self.video_out_width)                
              
                frames.append(stitched_frame)
                
                cnvc=cv2.cvtColor(stitched_frame,cv2.cv2.COLOR_RGB2BGR)
                
                

                if self.display:
                    #Show the output images
                    cv2.imshow('output', cnvc)
                    if cv2.waitKey(1) & 0xFF == ord("q"):
                        break
           
        cv2.destroyAllWindows()
        print('[INFO]: Video stitching finished')

        # Save video
        print('[INFO]: Saving {} in {}'.format(self.video_out_path.split('/')[-1],
                                               os.path.dirname(self.video_out_path)))
        #print("fps: ", fps)
        clip = ImageSequenceClip(frames, fps=fps)
 
        clip.write_videofile(self.video_out_path, codec ='mpeg4', audio=False,
                             verbose=False, bitrate="50000k")
        print('[INFO]: {} saved'.format(self.video_out_path.split('/')[-1]))


# Example call to 'VideoStitcher'
#for i in range(1,101):
    
    
# Example call to 'VideoStitcher'
rootDir ="videos/"     
# Example call to 'VideoStitcher'q
stitcher = VideoStitcher(left_video_in_path= rootDir + "test1.avi",
                         right_video_in_path=rootDir + "test2.avi",
                         left_video_in_path2=rootDir + "test3.avi",
                         right_video_in_path2=rootDir + "test4.avi",
                         video_out_path=rootDir + "testResult.avi")
                         #

stitcher.run()


In [ ]:
!pip3 freeze > requirements.txt